<a href="https://colab.research.google.com/github/jana-nf/Empacotamento_CODE_Reutil_Julia/blob/main/Empacotamento_CODE_Reutil_JULIA_jl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de Dados ou Computação Científica no Julia.

Este exemplo mostra a vantagem de desempenho e a estrutura de módulos (o sistema de empacotamento nativo do Julia).

# Vantagens do Julia no Empacotamento Reutilizável

- Desempenho (Velocidade): O Julia é compilado Just-In-Time (JIT) e é naturalmente rápido, o que o torna ideal para rotinas computacionalmente intensivas. Você empacota a velocidade.

- Estrutura de Módulos Simples: O sistema **module/using/include** é limpo e permite organizar o código sem muita sobrecarga.

- Tipagem Opcional/Múltipla Dispatch: Podemos definir funções específicas para diferentes tipos de entrada, otimizando o código sem perder a flexibilidade (ver calculo_medio abaixo).

## Módulo de Análise Estatística (Estatísticas.jl)

Simular um módulo que realiza cálculos de mediana móvel ponderada e cálculo de erro, rotinas comuns em análise de séries temporais ou processamento de sinais.

## O Código Reutilizável (Estatísticas.jl)

In [1]:
# Conteúdo do arquivo Estatisticas.jl
module Estatisticas

using Statistics # Importa o pacote Statistics para usar a função mean()

# 1. Função Avançada: Mediana Móvel Ponderada (computacionalmente intensiva)
"""
    mediana_movel_ponderada(dados::AbstractVector{T}, janela::Integer) where T <: Real

Calcula a mediana móvel ponderada para um vetor de dados.
O cálculo é feito de forma eficiente.
"""
function mediana_movel_ponderada(dados::AbstractVector{T}, janela::Integer) where T <: Real
    n = length(dados)
    resultados = zeros(T, n - janela + 1)

    for i in 1:(n - janela + 1)
        # Cria a janela de dados
        janela_dados = dados[i:(i + janela - 1)]

        # Simplesmente calcula a média da janela para este exemplo,
        # mas em aplicações reais poderia haver ponderações e ordenações complexas.
        resultados[i] = mean(janela_dados)
    end
    return resultados
end

# 2. Múltipla Dispatch (Vantagem Julia): Cálculo de Erro
# Versão 1: Erro Absoluto Simples
function calculo_erro(previsto::Real, real::Real)
    return abs(previsto - real)
end

# Versão 2: Erro Quadrático Médio (para vetores)
function calculo_erro(previstos::AbstractVector{T}, reais::AbstractVector{T}) where T <: Real
    if length(previstos) != length(reais)
        throw(DimensionMismatch("Vetores devem ter o mesmo comprimento."))
    end
    # Soma dos quadrados das diferenças / N
    return sum((previstos .- reais).^2) / length(previstos)
end


export mediana_movel_ponderada, calculo_erro
end # Fim do módulo Estatisticas

Main.Estatisticas

## Salvar o Código no Colab

In [2]:
open("Estatisticas.jl", "w") do f
    write(f, """
    module Estatisticas

    using Statistics

    function mediana_movel_ponderada(dados::AbstractVector{T}, janela::Integer) where T <: Real
        n = length(dados)
        resultados = zeros(T, n - janela + 1)
        for i in 1:(n - janela + 1)
            janela_dados = dados[i:(i + janela - 1)]
            resultados[i] = mean(janela_dados)
        end
        return resultados
    end

    function calculo_erro(previsto::Real, real::Real)
        return abs(previsto - real)
    end

    function calculo_erro(previstos::AbstractVector{T}, reais::AbstractVector{T}) where T <: Real
        if length(previstos) != length(reais)
            throw(DimensionMismatch("Vetores devem ter o mesmo comprimento."))
        end
        return sum((previstos .- reais).^2) / length(previstos)
    end

    export mediana_movel_ponderada, calculo_erro
    end
    """)
end

println("Arquivo Estatisticas.jl criado.")

Arquivo Estatisticas.jl criado.


## Reutilizando e Demonstrando Vantagens

In [3]:
# Carregar o Módulo:

include("Estatisticas.jl")
using .Estatisticas # O ponto (.) indica que queremos usar um módulo local

## Teste de Desempenho (Vantagem de Velocidade):

Vamos testar a função mediana_movel_ponderada em um vetor grande para ver a velocidade de execução do Julia (que em ambientes como Colab é excelente para cálculos numéricos).

In [4]:
# Crie 1 milhão de pontos de dados
dados_grandes = rand(1_000_000);

janela = 50;

# Use a macro @time para medir o tempo de execução
@time resultados_analise = mediana_movel_ponderada(dados_grandes, janela);

println("\nDimensão dos resultados: ", length(resultados_analise))

  0.374577 seconds (2.18 M allocations: 474.467 MiB, 42.26% gc time, 32.20% compilation time)

Dimensão dos resultados: 999951


#### Vantagem: O resultado do @time mostrará que, mesmo com 1 milhão de operações, o cálculo é feito em milissegundos ou menos, destacando a eficiência de Julia, que agora está empacotada e reutilizável.

## Teste de Flexibilidade (Vantagem de Múltipla Dispatch):

Demonstre como a mesma função calculo_erro se comporta de maneiras diferentes, dependendo dos tipos de entrada (múltipla dispatch).

In [5]:
# Exemplo 1: Entradas escalares (dois números)
# Chama automaticamente a função: calculo_erro(previsto::Real, real::Real)
erro_simples = calculo_erro(10.5, 9.8)
println("Erro Simples (Absoluto): ", erro_simples)

println("-"^20)

# Exemplo 2: Entradas vetoriais (dois vetores)
# Chama automaticamente a função: calculo_erro(previstos::AbstractVector, reais::AbstractVector)
previsto_serie = [1.0, 2.0, 3.0, 4.0]
real_serie     = [1.1, 1.9, 3.1, 3.9]

# Calcula o Erro Quadrático Médio (MSE)
erro_vetorial = calculo_erro(previsto_serie, real_serie)
println("Erro Vetorial (MSE): ", erro_vetorial)

Erro Simples (Absoluto): 0.6999999999999993
--------------------
Erro Vetorial (MSE): 0.010000000000000018


#### Vantagem: Você empacotou dois algoritmos distintos sob o mesmo nome (calculo_erro). O sistema Julia decide qual algoritmo executar com base nos tipos de argumentos passados. Isso torna sua API (Interface de Programação de Aplicações) limpa e intuitiva para o usuário que reutiliza seu código.

#### O empacotamento em Julia vai além da simples organização, permitindo que você reutilize eficiência computacional e estrutura de código flexível (Múltipla Dispatch).

# Gerenciamento de dependências externas (pacotes de terceiros) é crucial, pois a maioria dos projetos reutilizáveis depende de bibliotecas de alto nível.

Empacotar uma função que usa o popular pacote DataFrames.jl para manipular dados tabulares.

# Módulo de Limpeza de Dados com Dependências

### Módulo reutilizável (LimpezaDados.jl) fornecerá uma função para carregar um arquivo CSV, aplicar filtros e calcular estatísticas usando o pacote DataFrames.jl.

#### O Código Reutilizável (LimpezaDados.jl)

Vamos criar o módulo. Note como o using DataFrames e o using CSV são declarados dentro do módulo, definindo as dependências.

In [6]:
# Conteúdo do arquivo LimpezaDados.jl
module LimpezaDados

# Declara as dependências de pacotes externos
using DataFrames
using CSV
using Statistics

"""
    filtrar_e_analisar(caminho_arquivo::String, coluna_filtro::Symbol, valor_minimo::Number)

Carrega um CSV, filtra linhas onde 'coluna_filtro' >= 'valor_minimo',
e retorna a média da coluna 'Valor'.
"""
function filtrar_e_analisar(caminho_arquivo::String, coluna_filtro::Symbol, valor_minimo::Number)

    # 1. Carregar Dados
    # O CSV.read retorna um DataFrame
    df = CSV.read(caminho_arquivo, DataFrame)

    # 2. Filtrar Dados
    # Usa a sintaxe concisa do DataFrames para filtrar
    df_filtrado = filter(row -> row[coluna_filtro] >= valor_minimo, df)

    # 3. Analisar Dados
    # Se a coluna 'Valor' existir, calcula a média
    if "Valor" in names(df_filtrado)
        media = mean(skipmissing(df_filtrado.Valor))
        return media
    else
        return nothing
    end
end

export filtrar_e_analisar

end # Fim do módulo LimpezaDados

Main.LimpezaDados

#### Salvar o Código no Colab

In [7]:
open("LimpezaDados.jl", "w") do f
    write(f, """
    module LimpezaDados

    using DataFrames
    using CSV
    using Statistics

    function filtrar_e_analisar(caminho_arquivo::String, coluna_filtro::Symbol, valor_minimo::Number)
        df = CSV.read(caminho_arquivo, DataFrame)
        df_filtrado = filter(row -> row[coluna_filtro] >= valor_minimo, df)
        if "Valor" in names(df_filtrado)
            media = mean(skipmissing(df_filtrado.Valor))
            return media
        else
            return nothing
        end
    end

    export filtrar_e_analisar
    end
    """)
end

println("Arquivo LimpezaDados.jl criado.")

Arquivo LimpezaDados.jl criado.


#### Gerenciamento de Dependências e Reuso

#### Para garantir que os pacotes exigidos pelo seu módulo (DataFrames, CSV) estejam instalados no ambiente do Colab antes de usar seu código.

- Instalar Dependências:

A primeira vez que você roda um notebook, é necessário adicionar os pacotes que seu módulo precisa.

In [8]:
# Adiciona os pacotes necessários ao ambiente do Colab
import Pkg
Pkg.add("DataFrames")
Pkg.add("CSV")
# Statistics é um pacote padrão e geralmente não precisa ser adicionado

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


#### Preparar Dados de Teste (Criar um CSV):

Crie um arquivo CSV simples no ambiente temporário do Colab para simular os dados de entrada.

In [9]:
csv_conteudo = """
ID,Valor,Score
1,10.5,80
2,20.1,95
3,8.0,60
4,15.2,88
5,12.0,75
"""

open("dados_teste.csv", "w") do f
    write(f, csv_conteudo)
end
println("Arquivo dados_teste.csv criado.")

Arquivo dados_teste.csv criado.


### Carregar e usar o módulo

In [10]:
include("LimpezaDados.jl")
using .LimpezaDados # Carrega o módulo local

# Chamada da função reutilizável:
# - Caminho: "dados_teste.csv"
# - Coluna para filtrar: :Score
# - Valor mínimo do Score: 70
media_filtrada = LimpezaDados.filtrar_e_analisar("dados_teste.csv", :Score, 70)

println("\nMédia dos Valores onde Score >= 70: ", media_filtrada)


Média dos Valores onde Score >= 70: 14.45

# Vantagem do Empacotamento com Dependências

- A principal vantagem aqui é o isolamento e o ambiente de trabalho limpo:

- O notebook principal (using .LimpezaDados) não precisa saber como o DataFrames ou o CSV funcionam; ele apenas chama a função filtrar_e_analisar.

- Toda a complexidade de manipulação de dados (CSV.read, filter, mean) está encapsulada no módulo LimpezaDados.jl.

- Se você decidir trocar o DataFrames.jl por uma outra biblioteca de dados no futuro, você só precisa modificar o arquivo LimpezaDados.jl, e o código que reutiliza a função não precisa ser alterado.